In [ ]:
from lib.llm.model import model_api_client, make_impact_from_news, make_reasons_from_news
from pyspark.sql.functions import lit
from lib.scraping.scrap import extract_text_to_dataframe


In [ ]:
client = model_api_client()
news_data = spark.sql("select * from hive_metastore.default.news_data WHERE Date >= DATE_SUB(CURRENT_DATE(), 7) AND Date <= CURRENT_DATE()")
stock_df = spark.sql("select * from hive_metastore.default.stock_data WHERE stockID is not NULL")
countries = 'USA'
stock_df = stock_df.withColumn("Countries", lit(countries))
filtered_data = news_data.join(stock_df, (news_data.Actor1CountryCode == stock_df.Countries) | (news_data.Actor2CountryCode == stock_df.Countries)).toPandas()
news_df_processed = extract_text_to_dataframe(filtered_data.tail(10), filtered_data.columns[10], output_column = 'NewsBody')

In [ ]:
news_content = news_df_processed.loc[0,'NewsBody']
company_name = news_df_processed.loc[0,'company_name']
position = news_df_processed.loc[0,'position']

In [ ]:
impact = make_impact_from_news(news_content, company_name, position)
reasons = make_reasons_from_news(news_content, impact, company_name)